# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import os
import pandas as pd
os.chdir("C:/Users/Admin/Downloads/ЯНДЕКС/Проекты из курса")
data = pd.read_csv('predobrabotka_dan.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Таблица с различными типами данных в столбцах. 12 столбцов и 21525 строк.

## Шаг 2. Предобработка данных

### Обработка пропусков

**Поиск пропущенных значений методом isna(). Выведены первые строки для оценки пропусков.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


**Пропущенные значения в столбце days_employed соответствуют пропущенным значениям в столбце total_income. Можно предположить, что люди не имеющие дохода не имеют опыта работы(и наоборот), следовательно заменить отсутсвующие значения в обоих стобцах на ноль. Использован метод fillna() для заполнения пропусков.**

Чтобы проверить это соответствие, нужно отдельно посчитать количество строк, где одноврменно есть пропуски в обоих столбцах:

In [6]:
data[(data['days_employed'].isna()) & (data['total_income'].isna())].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

In [7]:
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [8]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

In [9]:
days_median = data.groupby('income_type')['days_employed'].median()

# для каждого типа занятости из серии, составленной выше
for income_type in days_median.index: 
# в нашем датафрейме мы выбираем только те строки, где тип занятости соответствует выбранному выше (по очереди), и берем из доход
    data.loc[data['income_type'] == income_type,'days_employed'] =  data.loc[data['income_type'] == income_type,'days_employed'].fillna(days_median.loc[income_type]) 
# и приравниваем их к самим себе (сохраняем), только заполняя пропуски доходом из таблички со средним доходом


In [10]:
income_median = data.groupby('income_type')['total_income'].median()

# для каждого типа занятости из серии, составленной выше
for income_type in days_median.index: 
    data.loc[data['income_type'] == income_type,'total_income'] =  data.loc[data['income_type'] == income_type,'total_income'].fillna(days_median.loc[income_type]) 



In [11]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Замена типа данных

**При анализе (data.info()) таблицы обнаружены вещественные значения в стобцах 'days_employed' и 'total_income'.
В теории рассматривали два метода изменения типов данных: 
    метод to_numeric() - при переводе все числа будут иметь тип данных float. 
    метод astype() - переводит в указанный тип значения. 
Так как переводим в целое число, используем метод astype() с аргументом 'int'**

In [12]:
#замена вещественных значений в столбцах 'days_employed' и 'total_income' на целочисленные
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info() #проверяем, что замена прошла успешно


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


In [13]:
data.head() #еще одна проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

- заменен вещественный тип данных на целочисленный (столбцы 'days_employed' и 'total_income')
- есть пояснение, какой метод используется для изменения типа данных и почему

### Обработка дубликатов

**При беглом анализе таблицы уже заметно наличие дубликатов в столбце 'education', отобразить их можно методом unique() или value_counts().**

In [14]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [15]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

**Дубликаты можем устранить приведением всех строк в столбце к единому регистру. Используем str.lower().**

In [16]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Проверим наличие дубликатов в столбцах, необходимых для предоставления ответов в дальнейшем. Это столбцы 'family_status', 'income_type', 'purpose'**

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates().reset_index(drop=True) 
data.duplicated().sum() #проверка

0

**Вывод**

- удалены дубликаты;
- есть пояснение, какой метод используется для поиска и удаления дубликатов (комбинирование value_counts() и str.lower());
- описаны возможные причины появления дубликатов в данных (использование разных регистров при вводе значения);

### Лемматизация

**В предыдущем пункте мы обнаружили скрытые дубликаты в столбце целей кредита. Повторно выведем список методом unique(). Напишем функцию которая возвращает значения, соответвующие лемме. Добавим столбец 'purpose_def' с указанем общей категории.**

In [19]:
from pymystem3 import Mystem
m = Mystem()
joint_data_purpose = ' '.join(data['purpose'])
lemmas = m.lemmatize(joint_data_purpose)

In [20]:
from collections import Counter
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

In [ ]:
purpose_list = data['purpose'].unique()
print(purpose_list)
#Делаем список
def purpose_change(purpose):
    lemma = m.lemmatize(purpose)
    if 'автомобиль' in lemma:
        return 'автомобиль'    
    #elif 'ремонт' in lemma:
        #return 'ремонт'       
    #elif 'строительство' in lemma:
        #return 'строительство'
    elif 'образование' in lemma:
        return 'образование'
    elif 'свадьба' in lemma:
        return 'свадьба'
    elif ('жилье' in lemma) | ('недвиж' in lemma) | ('ремонт' in lemma) | ('строительство' in lemma):
        return 'недвижимость и строительство'
    else:
        return 'другое'

    
data['purpose_def']= data['purpose'].apply(purpose_change) 

#проверяем
display(data.head(10))


['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Вывод**

**Основные цели получения кредита можно сгруппировать в группы:**
- автомобиль
- ремонт
- строительство
- образование
- свадьба
- жилье
- другое

### Категоризация данных

**Для ответа на последующие вопросы, проведена категоризация столбцов 'children' и 'total_income'. Для этого написана функция, которая возвращает наименование категории.**

In [ ]:
def get_children(qty):
    if qty == 0:
        return 'нет детей'
    return 'есть дети'

data['children_status'] = data['children'].apply(get_children)
data

In [ ]:
data['total_income'].max()

In [ ]:
data['total_income'].median()

In [ ]:
def get_income(qty):
    if qty <= 100000:
        return 'низкий уровень дохода'
    if qty <= 200000:
        return 'средний уровень дохода'
    return 'высокий уровень дохода'

data['income_level'] = data['total_income'].apply(get_income)
data

**Вывод**

**Добавлены категории есть дети/нет детей, а также уровень дохода разбит на три категории: высокий, средний, низкий.**

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#проверяем значения в столбце 'children'
data['children'].value_counts()

**76 человек с количеством детей 20 похоже на ошибку, так как происхождение ошибки не понятно, заменю на среднее значение количества детей
количество детей -1 похоже на опечатку, заменяем на 1**

In [ ]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 1
#children_mean = data['children'].mean()
data['children'].value_counts() #проверяю, что замена произошла успешно


**Произведено две группировки:**
1. по принципу наличия/отсутствия детей
2. по количеству детей

In [ ]:
children_grouped2 = data.groupby(['children_status', 'debt']).agg({'debt':'count'})
children_grouped2['rate'] = (children_grouped2['debt']/
                                   (children_grouped2.groupby('children_status')['debt'].transform('sum')))
children_grouped2['rate'] = children_grouped2['rate'].apply(lambda x: format(x, '.2%'))
children_grouped2

**Процент возврата кредита в группе 'нет детей' выше. Можно сделать вывод, что отсутствие детей увеличивает вероятность возврата кредита в срок.**

**Анализ влияния количества детей (не только их наличия), на вероятность возврата кредита в срок:**

In [ ]:
children_grouped = data.groupby(['children', 'debt',]).agg({'debt':'count'})
children_grouped['rate'] = (children_grouped['debt']/
                                   (children_grouped.groupby('children')['debt'].transform('sum'))).apply(lambda x: format(x, '.2%'))
children_grouped

**- Процент возврата в группах людей с одим ребенком и с двумя почти одинаковый. Можно сделать вывод, что само количество (один или два ребенка) не влияет на вероятность возрата кредита в срок.**

**- Данных по количеству детей 3, 4 и 5 мало для проведения аналогичной аналитики.**

**Вывод**

**Отсутствие детей увеличивает вероятность возврата кредита в срок**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_grouped = data.groupby(['family_status_id', 'debt', 'family_status']).agg({'debt':'count'})
family_grouped['rate'] = (family_grouped['debt']/
                                   (family_grouped.groupby('family_status_id')['debt'].transform('sum'))).apply(lambda x: format(x, '.2%'))
family_grouped


**Вывод**

**Наибольший процент возврата кредита в срок наблюдается в группе вдовец / вдова, в разводе, женат / замужем. По предоставленным данным можно сделать вывод, что наиболее внушающими доверие для банков должны быть вдовы и вдовцы, и наименее - одинокие люди, не состоявшие в официальном браке. Рейтинг выглядит следующим образом:**

In [ ]:
family_grouped.sort_values(by='rate', ascending=False)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_level = data.groupby(['income_level', 'debt']).agg({'debt':'count'})
income_level['rate'] = (income_level['debt']/
                                   (income_level.groupby('income_level')['debt'].transform('sum'))).apply(lambda x: format(x, '.2%'))
income_level.sort_values(by='rate', ascending=False)


**Вывод**

**Возврат кредита в срок менее вероятен у людей в категории "средний уровень дохода" (от 100 до 200)**

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_grouped = data.groupby(['purpose_def', 'debt']).agg({'debt':'count'})
purpose_grouped['rate'] = (purpose_grouped['debt']/
                                   (purpose_grouped.groupby('purpose_def')['debt'].transform('sum'))).apply(lambda x: format(x, '.2%'))
purpose_grouped.sort_values(by='rate', ascending=False)

**Вывод**

**Предпочтительными целями получения кредита для банка должны быть цели, связанные с ремонтом и недвижимостью, наименее предпочтительные - связанные с образованием и авто.**

## Шаг 4. Общий вывод

**Идеальный кандидат: вдова/вдовец, без детей, с высоким уровенем дохода, затеявший ремонт. Портрет нежелательного: не женат/не замужем, с детьми, средний уровень дохода, заинтересовавшийся образованием либо действиями связанными с автомобилем.**